# Q-러닝 기초: 미로에서 길 찾기

- `E`에서 `G`에 도착하는 가장 빠른 길 찾기

```
--------------------
| A     B    C | D  
-----               
  E |   F  | G   H |
|          ----    |
| I     J    K   L |
-----------    -----
```

## 1. 환경 구성

- 상태, 행동, 보상 정의

1. 모델의 입력 정의

2. 수행될 수 있는 행동(=모델의 출력) 정의

3. 보상 정의

### 1-1. 상태

- 에이전트는 `A`~`L` 위치 중 임의의 한 곳에 있을 수 있음

In [1]:
env = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'H': 7,
    'I': 8,
    'J': 9,
    'K': 10,
    'L': 11,
}

### 1-2. 행동

- 에이전트는 임의 위치 `X`에서 다른 위치 `Y`로 이동할 수 있음

In [2]:
actions = [i for i in range(len(env))]
actions

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

### 1-3. 보상

- 에이전트는 행동을 취할 때 벽을 넘을 수 없음

  - 벽을 넘는 경우 보상에 0을 부여

- 특정 위치에서 다른 위치로 이동하는 모든 경우에 대한 보상 행렬을 작성


In [3]:
rewards = [[0] * len(env) for _ in range(len(env))]

rewards[env['A']][env['B']] = 1

rewards[env['B']][env['A']] = 1
rewards[env['B']][env['C']] = 1
rewards[env['B']][env['F']] = 1

rewards[env['C']][env['B']] = 1
rewards[env['C']][env['G']] = 1

rewards[env['D']][env['H']] = 1

rewards[env['E']][env['I']] = 1

rewards[env['F']][env['B']] = 1
rewards[env['F']][env['J']] = 1

rewards[env['G']][env['C']] = 1
rewards[env['G']][env['H']] = 1

rewards[env['H']][env['D']] = 1
rewards[env['H']][env['G']] = 1
rewards[env['H']][env['L']] = 1

rewards[env['I']][env['E']] = 1
rewards[env['I']][env['J']] = 1

rewards[env['J']][env['I']] = 1
rewards[env['J']][env['F']] = 1
rewards[env['J']][env['K']] = 1

rewards[env['K']][env['J']] = 1
rewards[env['K']][env['L']] = 1

rewards[env['L']][env['K']] = 1
rewards[env['L']][env['H']] = 1

# 최종 목적지에 가장 높은 보상 부여
rewards[env['G']][env['G']] = 1000

rewards

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 1000, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]

## 2. 모델 구성

### 2-1. 개념 정리

#### 2-1-1. Q-value

> 특정 상태에서 특정 행동을 취했을 때 에이전트가 기대할 수 있는 미래 보상의 총합
>
> $Q:(s \in S,  a \in A) ➝ Q(s, a) \in R$

$Q(s, a)$는 상태 $s$에서 행동 $a$를 수행할 때의 $Q-value$

#### 2-1-2. 시간 차(time difference)

> 현재 행동의 결과로 얻은 보상과, 미래 행동의 Q값 간의 차이
>
> $TD(s_t, a_t) = R(s_t, a_t) = \gamma \max(Q(s_{t+1}, a)) - Q(s_t, a_t)$

- $Q-value$의 의미를 이해하기 위해 필요한 개념



$TD(s_t, a_t)$는 상태 $s_t$에서 행동 $a_t$를 수행함으로써 얻는 보상 $R(s_t, a_t)$에, 미래 상태 $s_{t+1}$에서 수행된 최고 행동의 $Q-value$에 할인 계수 $\gamma \in [0, 1]$에 의해 할인된 값을 더한 값

- 모델은 더 나은 보상을 얻으려 하므로 시간차가 가장 큰 경우를 찾으려 할 것임

- 모델이 받는 보상이 크면 특정 $Q(s, a)$ 값이 증가

- 모델은 보상뿐만이 아니라 높은 Q값을 추구해야 함 - $\gamma \max(Q(s_{t+1}, a))$

- 어떤 시점에서 모델은 높은 Q값으로 이어지는 모든 전이를 알게 될 것이며, 이 전이들의 Q값은 시간이 지남에 따라 이미 증가

  - 결국 시간차는 감소하며, 최종 목표에 가까워질수록 시간차는 줄어듦

➡️ 즉 모델은 높은 Q값을 추구하며 최종적으로 낮은 시간차를 추구함

#### 2-1-3. 벨만 방정식

> Q값을 업데이트하는 방법
> 
> $Q_t(s_t, a_t) = Q_{t-1}(s_t, a_t) + \alpha TD_t(s_t, a_t)$

- $\alpha \in [0, 1]$은 학습률


#### 2-1-4. Q러닝의 목표

- 더이상 업데이트되지 않을 때까지 특정 반복 횟수에 걸쳐 Q값을 업데이트하는 것

### 2-2. 모델 초기화

In [4]:
from random import randint
import numpy as np

gamma = 0.9
alpha = 0.75

state = 4
reward = 0

q_table = [[0 for _ in range(len(actions))] for _ in range(len(env))]

In [5]:
for _ in range(1000):
    possible_actions = []
    for i in range(len(rewards[state])):
        if rewards[state][i] > 0:
            possible_actions.append(i)

    next_state = np.random.choice(possible_actions)
    next_reward = rewards[state][next_state]

    time_diff = next_reward + gamma * q_table[next_state][np.argmax(q_table[next_state])] - q_table[state][next_state]

    q_table[state][next_state] = q_table[state][next_state] + alpha * time_diff

    state = next_state

In [7]:
nodes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

current = env['E']

counter = 0
while True:
    counter += 1

    if counter > 20:
        print("...")
        break

    print(nodes[current], end="")

    if current == env['G']:
        break

    next_ = np.argmax(q_table[current])

    if(np.max(q_table[current])) == 0:
        break

    current = next_

    print(' > ', end="")


E > I > J > F > B > C > G